# Clase 7 - Clasificador Bayesiano

## Carga de Librerias

Los clasificadores Naive Bayes (Naive Bayes Classifier - NBC) se usan, como su nombre indica, para problemas de clasificación, y en concreto se pueden aplicar para texto.

En este ejemplo vamos a implementar un modelo NBC a un dataset de un portal de Noticias muy famoso en España. Cada usuario comparte un link a una noticia y le puede asignar una categoría.

In [4]:
import pandas as pd

noticias = pd.read_csv("noticias.csv").iloc[:2000,]
noticias

,descripcion,categoria
0,"Aunque parezca mentira, las emisiones de dióxi...",cultura
1,Hubo un proyecto impulsado por la Unión Europe...,cultura
2,China ha confirmado la conclusión con éxito de...,tecnología
3,"En su fructífera carrera como humorista, actor...",cultura
4,Tras dos años de negociación entre la instituc...,cultura
...,...,...
1995,Sasha Shulgin estuvo en España y lo contó en s...,cultura
1996,El fabricante Xiaomi acaba de lanzar su Mi Not...,tecnología
1997,"El faro de la isla de Buda, en el delta del Eb...",cultura
1998,Los dinosaurios florecieron en Europa hasta el...,cultura


La variable objetivo es categoria y la variable independiente es descripcion que contiene la descripcion de la noticia

In [5]:
print(noticias.categoria.value_counts())


cultura       1029
tecnología     532
ocio           439
Name: categoria, dtype: int64


**EJEMPLO DESBALANCEADO, ES QUE EL 50% DE LA INFORMACION ESTA AGRUPADA EN UNA SOLA CATEGORIA**

Vemos que hay noticias de 3 tipos de categorías distintas.

Los clasificadores Naive Bayes esperan como input un vector, así que para poder entrenarlos tenemos que vectorizar el texto. Para ello una buena opción es usar vectorización Tf-IDF


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

Vamos a modificar nuestro vectorizador añadiendole dos parámetros:
    
**ELIMINIAR STOPWORDS**

Stopwords (palabras vacías) son palabras que no tienen ningún contenido semántico. 

Por ejemplo, en la frase el perro ladra el artículo el no aporta ningún valor a la frase. Me he descargado una lista de stopwords de castellano de este repositorio en Github.


In [7]:
import json
#SE CARGA LAS PALABRAS QUE NO TIENEN VALOR INFORMATIVO
with open("stopwords-es.json") as fname:
    stopwords_es = json.load(fname)
#SE IMPRIMEN LAS PRIMERAS 30 STOPWORDS
print(stopwords_es[:30])

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '_', 'a', 'actualmente', 'acuerdo', 'adelante', 'ademas', 'ademÃ¡s', 'adrede', 'afirmÃ³', 'agregÃ³', 'ahi', 'ahora', 'ahÃ\xad', 'al', 'algo', 'alguna', 'algunas', 'alguno', 'algunos', 'algÃºn']


**ELIMINAR ACENTOS**

También vamos a eliminar los acentos de las palabras, ésto tiene una ventaja, y es que si en el conjunto de datos no tenemos confianza en la calidad de los escritores, al eliminar los acentos evitamos que si un escritor no usa acentos no considere sus palabras como palabras distintas.

En castellano, esto tiene un problema, y es que hay palabras con significado distinto que sólo se diferencian por la existencia de una tilde (se llaman palabras con acento diacrítico (por ejemplo, de y dé). Asumimos pues que el impacto de estas palabras no es muy alto.

In [8]:
#SE ELIMINAN ACENTOS 
#POR DEFAULT CONVIERTE A MINUSCULAS Y LUEGO COMPARA
vectorizador = TfidfVectorizer(strip_accents="unicode", stop_words=stopwords_es)

In [9]:
#SE IMPRIME EL TAMAÑO DE NOTICIAS
print(noticias.shape)

(2000, 2)


In [12]:
#SE APLICA EL VECTORIZADOR A LAS NOTICIAS
vectorizador.fit_transform(noticias.descripcion)

<2000x17548 sparse matrix of type '<class 'numpy.float64'>'
	with 46857 stored elements in Compressed Sparse Row format>

Dicha matriz nos indica que tenemos 2000 artículos que tienen 46857 palabras distintas (sin contar acentos o stopwords)

In [14]:
# Dicha matriz nos indica que tenemos 2000 artículos que tienen 46857 palabras distintas 
# (sin contar acentos o stopwords)
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
# Dado que los vectorizadores devuelven una matriz sparse (escasa) creamos un transformador 
# que las convierta a densas
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer
from scipy.sparse import issparse
# http://rasbt.github.io/mlxtend/
class DenseTransformer(BaseEstimator):
    def __init__(self, return_copy=True):
        self.return_copy = return_copy
        self.is_fitted = False

    def transform(self, X, y=None):
        if issparse(X):
            return X.toarray()
        elif self.return_copy:
            return X.copy()
        else:
            return X

    def fit(self, X, y=None):
        self.is_fitted = True
        return self

    def fit_transform(self, X, y=None):
        return self.transform(X=X, y=y)

### Clasificadores

Scikit-learn tiene tres implementaciones del clasificador **Naive Bayes**, **GaussianNB**, **BernoulliNB** y **MultinomialNB**, y cada una se diferencia por como calcula las probabilidades de que cada elemento aparezca en los datos.

In [15]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

**GaussianNB** asume que los datos siguien una distribución Gausiana

Se define el pipeline

In [16]:
pipeline_gaussiano = make_pipeline(
    #SE APLICA EL VECTORIZADOR
    vectorizador,
    #SE CONVIERTE ARRAY
    DenseTransformer(),
    #SE APLICA EL CLASIFICADOR GAUSSIANO
    GaussianNB()
)

In [19]:
#SE CREA EL MODELO DANDO LOS NOMBRES DE LAS NOTICIAS Y LAS CATEGORIAS
#UTILIZANDO EL PIPELINE QUE TIENE EL VECTORIZADOR Y LAS DEMAS TRANSFORMACIONES
pipeline_gaussiano.fit(X=noticias.descripcion, y=noticias.categoria);

In [20]:
#SE IMPRIMEN LAS PREDICCIONES 
pipeline_gaussiano.predict(noticias.descripcion)

array(['cultura', 'cultura', 'tecnología', ..., 'cultura', 'cultura',
       'cultura'], dtype='<U10')

In [22]:
#SE IMPORTA EL MODULO F1 SCORE
from sklearn.metrics import f1_score

MARCA UN ERROR LA VALIDACION CRUZADA
cross_val_score(pipeline_gaussiano, noticias.descripcion, noticias.categoria, scoring=f1_score)

Vemos que la validación cruzada con la puntuación 
F1 nos da un error **ValueError: Target is multiclass but average='binary'. Please choose another average setting., esto es por que las medidas de clasificación tienen distintas maneras de calcularse en funcion de si es un caso de clasificación binaria (el parámetro por defecto) o clasificación multiclase.**

En concreto para el caso de la puntuación F1, nos permite los siguientes tipos de cálculos.

- binary: Devuelve la puntuación para la clase especificada en el argumento pos_label. Solo se puede aplicar en clasificación binaria. Este es el caso que vimos en el apartado de medidas de evaluación de modelos de clasificación.
- micro: Cuenta todos el número total de Verdaderos positivos (TP), Falsos Negativos (FN) y Falsos Positivos (FP) y calcula una precisión y sensibilidad total y obtiene el F1. Es mejor cuando hay clases no balanceadas (muchos casos más de una clase que de las demás).

**CLASES BALANCEADAS**
- macro: Calcula la precisión y sensibilidad media de cada clase, hace su media aritmética y calcula el parámetro F1.
- weighted: Calcula la precisión y sensibilidad media de cada clase, hace su media ponderada por el número de observaciones de cada clase y calcula el parámetro F1

In [24]:
#CALCULA EL VALOR DE F1
def f1_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return f1_score(y, preds, average="micro")

Como hemos visto antes, el vectorizador TF-IDF coge por defecto todas las palabras (removiendo palabras sin aporte de información). Podemos restringir la cantidad de palabras que considera con el parámetro max_features.

MAX FEATURES ES UN PARAMETRO QUE DICE QUE SE FIJE EN LAS PRIMERAS 1000 PALABRAS MAS FRECUENTES

In [28]:
pipeline_gaussiano = make_pipeline(
    TfidfVectorizer(strip_accents="unicode", stop_words=stopwords_es, max_features=1000),
    DenseTransformer(),
    GaussianNB()
)

In [32]:
#SE OBTIENEN LOS VALORES CRUZADOS
try:
  x = cross_val_score(pipeline_gaussiano, noticias.descripcion, noticias.categoria, scoring=f1_multietiqueta,cv=3)
except:
  pass
x
#ENTRE MAS CERCANO A UNO ES MEJOR

c:\Users\felip\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aa', 'aaon', 'adema', 'adia3', 'afirma3', 'agrega3', 'aha', 'ais', 'ala3', 'algaon', 'alla', 'amos', 'an', 'antaa', 'aoltima', 'aoltimas', 'aoltimo', 'aoltimos', 'aqua', 'as', 'asa', 'asegura3', 'bamos', 'ca3mo', 'comenta3', 'considera3', 'cua', 'da3nde', 'deja3', 'dema', 'despua', 'detra', 'estara', 'estuvia', 'explica3', 'expresa3', 'fua', 'haba', 'habra', 'hubia', 'indica3', 'informa3', 'is', 'lla', 'llas', 'llega3', 'llos', 'ma', 'manifesta3', 'menciona3', 'ndo', 'nes', 'ningaon', 'nta', 'ntas', 'nto', 'ntos', 'paa', 'podra', 'pra3ximo', 'pra3ximos', 'qua', 'queda3', 'quia', 'ramos', 'realiza3', 'sa', 'sa3lo', 'sas', 'segaon', 'semos', 'sos', 'sta', 'stas', 'ste', 'stos', 'tambia', 'tao', 'tena', 'tendra', 'todava', 'trava', 'tuvia'] not in stop_words.
 

array([0.54572714, 0.52023988, 0.48198198])

Las dos implementaciones de clasificadores NB más utilizadas para clasificación de texto son MultinomialNB (que asume que la distribución de probabilidades de las palabras en el conjunto de datos sigue una distribución multinomial y BernouilliNB , que asume que siguen una distribución de Bernouilli multivariable (donde la existencia de cada palabra se considera que es una variable binaria distinta).

Según la documentación, el clasificador MultinomialNB funciona bien con vectores TFIDF

**MultinomialNB**

In [33]:
#DISTRIBUCION MULTINOMIAL
pipeline_multinomial = make_pipeline(
    TfidfVectorizer(strip_accents="unicode", stop_words=stopwords_es, max_features=500),
    DenseTransformer(),
    MultinomialNB(),
)

In [34]:
#SE OBTIENE EL VALOR CRUZADO PARA UNA DISTRIBUCION MULTINOMIAL
print(cross_val_score(pipeline_multinomial, noticias.descripcion, noticias.categoria, scoring=f1_multietiqueta,cv=3))

c:\Users\felip\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aa', 'aaon', 'adema', 'adia3', 'afirma3', 'agrega3', 'aha', 'ais', 'ala3', 'algaon', 'alla', 'amos', 'an', 'antaa', 'aoltima', 'aoltimas', 'aoltimo', 'aoltimos', 'aqua', 'as', 'asa', 'asegura3', 'bamos', 'ca3mo', 'comenta3', 'considera3', 'cua', 'da3nde', 'deja3', 'dema', 'despua', 'detra', 'estara', 'estuvia', 'explica3', 'expresa3', 'fua', 'haba', 'habra', 'hubia', 'indica3', 'informa3', 'is', 'lla', 'llas', 'llega3', 'llos', 'ma', 'manifesta3', 'menciona3', 'ndo', 'nes', 'ningaon', 'nta', 'ntas', 'nto', 'ntos', 'paa', 'podra', 'pra3ximo', 'pra3ximos', 'qua', 'queda3', 'quia', 'ramos', 'realiza3', 'sa', 'sa3lo', 'sas', 'segaon', 'semos', 'sos', 'sta', 'stas', 'ste', 'stos', 'tambia', 'tao', 'tena', 'tendra', 'todava', 'trava', 'tuvia'] not in stop_words.
 

[0.66266867 0.68215892 0.67567568]


c:\Users\felip\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aa', 'aaon', 'adema', 'adia3', 'afirma3', 'agrega3', 'aha', 'ais', 'ala3', 'algaon', 'alla', 'amos', 'an', 'antaa', 'aoltima', 'aoltimas', 'aoltimo', 'aoltimos', 'aqua', 'as', 'asa', 'asegura3', 'bamos', 'ca3mo', 'comenta3', 'considera3', 'cua', 'da3nde', 'deja3', 'dema', 'despua', 'detra', 'estara', 'estuvia', 'explica3', 'expresa3', 'fua', 'haba', 'habra', 'hubia', 'indica3', 'informa3', 'is', 'lla', 'llas', 'llega3', 'llos', 'ma', 'manifesta3', 'menciona3', 'ndo', 'nes', 'ningaon', 'nta', 'ntas', 'nto', 'ntos', 'paa', 'podra', 'pra3ximo', 'pra3ximos', 'qua', 'queda3', 'quia', 'ramos', 'realiza3', 'sa', 'sa3lo', 'sas', 'segaon', 'semos', 'sos', 'sta', 'stas', 'ste', 'stos', 'tambia', 'tao', 'tena', 'tendra', 'todava', 'trava', 'tuvia'] not in stop_words.
 

Vemos que efectivamente, MultinomialNB parece funcionar mejor que GaussianNB para vectores TF-IDF

**Bernoulli**

Para el clasificador BernouilliNB se necesita tener los vectores de palabras cono vectores binarios (1 si la palabra existe o 0 si no), así que en vez de usar TfidfVectorizer en este caso usaremos el CountVectorizer pasandole el parámetro binary=True para que devuelva 1 ó 0 en vez del número de veces que aparece la palabra en la frase.

In [35]:
#PARA BERNOULLI SE UTILIZA UN VECTORIZADOR DE CONTEO
from sklearn.feature_extraction.text import CountVectorizer

#PROPIEDAD BINARY ES PARA QUE SOLO SE FIJE SI LA PALABRA ESTA O NO ESTA
vectorizador_count = CountVectorizer(stop_words=stopwords_es, binary=True, strip_accents="unicode", max_features=1000)

In [36]:
# Una funcionalidad interesante del CountVectorizer es que nos permite ver cuantas veces 
# aparece cada palabra en el corpus.
vectorizador_count.fit(noticias.descripcion)

c:\Users\felip\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aa', 'aaon', 'adema', 'adia3', 'afirma3', 'agrega3', 'aha', 'ais', 'ala3', 'algaon', 'alla', 'amos', 'an', 'antaa', 'aoltima', 'aoltimas', 'aoltimo', 'aoltimos', 'aqua', 'as', 'asa', 'asegura3', 'bamos', 'ca3mo', 'comenta3', 'considera3', 'cua', 'da3nde', 'deja3', 'dema', 'despua', 'detra', 'estara', 'estuvia', 'explica3', 'expresa3', 'fua', 'haba', 'habra', 'hubia', 'indica3', 'informa3', 'is', 'lla', 'llas', 'llega3', 'llos', 'ma', 'manifesta3', 'menciona3', 'ndo', 'nes', 'ningaon', 'nta', 'ntas', 'nto', 'ntos', 'paa', 'podra', 'pra3ximo', 'pra3ximos', 'qua', 'queda3', 'quia', 'ramos', 'realiza3', 'sa', 'sa3lo', 'sas', 'segaon', 'semos', 'sos', 'sta', 'stas', 'ste', 'stos', 'tambia', 'tao', 'tena', 'tendra', 'todava', 'trava', 'tuvia'] not in stop_words.
 

CountVectorizer(binary=True, max_features=1000,
                stop_words=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                            '_', 'a', 'actualmente', 'acuerdo', 'adelante',
                            'ademas', 'ademÃ¡s', 'adrede', 'afirmÃ³', 'agregÃ³',
                            'ahi', 'ahora', 'ahÃ\xad', 'al', 'algo', 'alguna',
                            'algunas', 'alguno', 'algunos', 'algÃºn', ...],
                strip_accents='unicode')

In [38]:
#FRECUENCIA DE CADA PALABRA
pipeline_bernouilli = make_pipeline(
    vectorizador_count,
    DenseTransformer(),
    BernoulliNB(),
)

print(cross_val_score(pipeline_bernouilli, noticias.descripcion, noticias.categoria, scoring=f1_multietiqueta))

c:\Users\felip\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aa', 'aaon', 'adema', 'adia3', 'afirma3', 'agrega3', 'aha', 'ais', 'ala3', 'algaon', 'alla', 'amos', 'an', 'antaa', 'aoltima', 'aoltimas', 'aoltimo', 'aoltimos', 'aqua', 'as', 'asa', 'asegura3', 'bamos', 'ca3mo', 'comenta3', 'considera3', 'cua', 'da3nde', 'deja3', 'dema', 'despua', 'detra', 'estara', 'estuvia', 'explica3', 'expresa3', 'fua', 'haba', 'habra', 'hubia', 'indica3', 'informa3', 'is', 'lla', 'llas', 'llega3', 'llos', 'ma', 'manifesta3', 'menciona3', 'ndo', 'nes', 'ningaon', 'nta', 'ntas', 'nto', 'ntos', 'paa', 'podra', 'pra3ximo', 'pra3ximos', 'qua', 'queda3', 'quia', 'ramos', 'realiza3', 'sa', 'sa3lo', 'sas', 'segaon', 'semos', 'sos', 'sta', 'stas', 'ste', 'stos', 'tambia', 'tao', 'tena', 'tendra', 'todava', 'trava', 'tuvia'] not in stop_words.
 

[0.6725 0.6475 0.64   0.6    0.6525]
